# Satellite Image Data Set

## Data Preprocessing
Missing Values - Replace by mean, Replace by Neighbourhood, Row-wise, Row Spectral Wise
Use Multi-class SVM with different kernels, Multi Class Learning
Stratified K-Fold validation for equal number of classes in each fold
https://archive.ics.uci.edu/ml/datasets/Statlog+(Landsat+Satellite)

## Read the input data

In [1]:
import numpy as np

def readInputFiles():
    rawTrainData = np.loadtxt('sat_noisy.trn',delimiter=',')
    rawTestData = np.loadtxt('sat-test.csv.dat',delimiter=',')
    # print rawTrainData.shape
    # print rawTestData.shape
    # print rawTrainData[0]
    YTrain = rawTrainData[:,-1]
    XTrain = rawTrainData[:,:-1]
    # print YTrain.shape
    # print XTrain.shape

    return XTrain, YTrain, rawTestData
 

## Handle Missing Values (NaN)

In [76]:

# Replacing NaN
# Replace by column mean, by zero, row-wise, row spectral-wise
import  scipy.stats as stats
def replaceMissingValues(X, replacement):
    func = None
    if replacement == 'colmean':
        func = stats.nanmean(X,axis=0)
    elif replacement == 'colmedian':
        func = stats.nanmedian(X,axis=0)
    elif replacement =='zeros':
        pass
    elif replacement =='rowmean':
        func = stats.nanmean(X,axis=1)
    elif replacement =='rowmedian':
        func = stats.nanmedian(X,axis=1)
    elif replacement =='rowspectralmean':
        XSpectrum = np.hsplit(X,4)
        XSpectrum1 = replaceMissingValues(XSpectrum[0], "rowmean")
        XSpectrum2 = replaceMissingValues(XSpectrum[1], "rowmean")
        XSpectrum3 = replaceMissingValues(XSpectrum[2], "rowmean")
        XSpectrum4 = replaceMissingValues(XSpectrum[3], "rowmean")
        func = np.hstack((XSpectrum1,XSpectrum2,XSpectrum3,XSpectrum4))
    elif replacement =='rowspectralmedian':
        XSpectrum = np.hsplit(X,4)
        XSpectrum1 = replaceMissingValues(XSpectrum[0], "rowmedian")
        XSpectrum2 = replaceMissingValues(XSpectrum[1], "rowmedian")
        XSpectrum3 = replaceMissingValues(XSpectrum[2], "rowmedian")
        XSpectrum4 = replaceMissingValues(XSpectrum[3], "rowmedian")
        func = np.hstack((XSpectrum1,XSpectrum2,XSpectrum3,XSpectrum4))
    elif replacement =='rowspectralmin':
        XSpectrum = np.hsplit(X,4)
        XSpectrum1 = replaceMissingValues(XSpectrum[0], "rowmin")
        XSpectrum2 = replaceMissingValues(XSpectrum[1], "rowmin")
        XSpectrum3 = replaceMissingValues(XSpectrum[2], "rowmin")
        XSpectrum4 = replaceMissingValues(XSpectrum[3], "rowmin")
        func = np.hstack((XSpectrum1,XSpectrum2,XSpectrum3,XSpectrum4))
    elif replacement =='rowspectralmax':
        XSpectrum = np.hsplit(X,4)
        XSpectrum1 = replaceMissingValues(XSpectrum[0], "rowmax")
        XSpectrum2 = replaceMissingValues(XSpectrum[1], "rowmax")
        XSpectrum3 = replaceMissingValues(XSpectrum[2], "rowmax")
        XSpectrum4 = replaceMissingValues(XSpectrum[3], "rowmax")
        func = np.hstack((XSpectrum1,XSpectrum2,XSpectrum3,XSpectrum4))
    elif replacement == 'colmin':
        func = np.nanmin(X,axis=0)
    elif replacement =='colmax':
        func = np.nanmax(X,axis=0)
    elif replacement == 'rowmin':
        func = np.nanmin(X,axis=1)
    elif replacement =='rowmax':
        func = np.nanmax(X,axis=1)
    elif replacement =='mid':
        pass
    
    inds = np.where(np.isnan(X))
    #X[inds]=np.take(func,inds[0])
    
    #print "X before replace", X[1][10]
    
    if replacement == 'colmean' or replacement == 'colmedian' or replacement == 'colmin' or replacement == 'colmax':
        X[inds[0],inds[1]] = func[inds[1]]
    elif replacement == 'rowmean' or replacement == 'rowmedian' or replacement =='rowmin' or replacement =='rowmax':
        X[inds[0],inds[1]] = func[inds[0]]
    elif replacement =='zeros':
        X[inds[0],inds[1]]= 0
    elif replacement =='mid':
        X[inds[0],inds[1]]= 127
        
    #print "X after replace", X[1][10]    
    #print "inds", (inds)
    #print "func", repr(func)
    return X

In [3]:
# #Multi Class SVM
# from sklearn import svm
# # clf = svm.SVC(decision_function_shape='ovo')
# # clf = svm.SVC(kernel='linear',probability=True)
# clf.fit(XTrain, YTrain) 
# # dec = clf.decision_function(XTest)
# # print dec.shape[1] # 6 classes: 6*5/2 = 15

# # clf.decision_function_shape = "ova"
# # dec = clf.decision_function(XTest)
# # dec = clf.predict(XTest)
# # print dec.shape[1] # 6 classes
# # YPred = clf.predict_proba(XTest)
# YPred = clf.predict(XTest)
# print YPred
# # predictedClasses =YPred.argmax(axis=1)
# # print predictedClasses

Try <br>
    K nearest neighbors <br>
    Multi class SVM <br>
    Multi class Perceptron <br>

In [83]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
def classifyOneVsRestClassifier(XTrain, XTest, YTrain, YTest):
    YPred = OneVsRestClassifier(LinearSVC(random_state=0)).fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
#     print diff
    score = diff[diff == 0].size
#     print score
#     print YPred.size
    return (100.0 * score)/(YPred.size)

In [84]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
def classifyOneVsOneClassifier(XTrain, XTest, YTrain, YTest):
    YPred = OneVsOneClassifier(LinearSVC(random_state=0)).fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [85]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
def classifyOutputCodeClassifier(XTrain, XTest, YTrain, YTest):
    clf = OutputCodeClassifier(LinearSVC(random_state=0),code_size=2, random_state=0)
    YPred = clf.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [86]:
from sklearn.ensemble import RandomForestClassifier
def classifyRandomForestClassifier(XTrain, XTest, YTrain, YTest):
    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(XTrain, YTrain)
    YPred = clf.predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [87]:
from sklearn.naive_bayes import GaussianNB
def classifyGaussianNaiveBayesClassifier(XTrain, XTest, YTrain, YTest):
    gnb = GaussianNB()
    YPred = gnb.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [88]:
from sklearn.naive_bayes import BernoulliNB
def classifyBernoulliNaiveBayesClassifier(XTrain, XTest, YTrain, YTest):
    bnb = BernoulliNB()
    YPred = bnb.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [89]:
from sklearn.naive_bayes import MultinomialNB
def classifyMultinomialNaiveBayesClassifier(XTrain, XTest, YTrain, YTest):
    mnb = MultinomialNB()
    YPred = mnb.fit(XTrain, YTrain).predict(XTest)
    diff = YPred - YTest
    score = diff[diff == 0].size
    return (100.0 * score)/(YPred.size)

In [90]:
from sklearn.cross_validation import KFold
def kFoldCrossVal(XTrain, YTrain, classify):
    n_folds=5
    kf = KFold(4435, n_folds)
    score = 0.0
    for train, test in kf:
        score += classify(XTrain[train,:], XTrain[test,:], YTrain[train], YTrain[test])

    return score/n_folds

In [94]:
scoreOneVsRestClassifier = []
scoreOneVsOneClassifier = []
scoreOutputCodeClassifier = []
scoreRandomForestClassifier = []
scoreGaussianNaiveBayesClassifier = []
scoreBernoulliNaiveBayesClassifier = []
scoreMultinomialNaiveBayesClassifier = []

In [95]:
def PreProcessClassificationScores(replacement):
    # Replace missing values (NaN) with replacement
    XTrain, YTrain, rawTestData = readInputFiles()
    XTrain = replaceMissingValues(XTrain, replacement)
    XTest = replaceMissingValues(rawTestData, replacement)
    #score = kFoldCrossVal(XTrain, YTrain, classifyOneVsRestClassifier)
    #print score
    score1 = kFoldCrossVal(XTrain, YTrain, classifyOneVsOneClassifier)
    print score1
    #score2 = kFoldCrossVal(XTrain, YTrain, classifyOutputCodeClassifier)
    #print score2
    score3 = kFoldCrossVal(XTrain, YTrain, classifyRandomForestClassifier)
    print score3
    score4 = kFoldCrossVal(XTrain, YTrain, classifyGaussianNaiveBayesClassifier)
    print score4
    score5 = kFoldCrossVal(XTrain, YTrain, classifyBernoulliNaiveBayesClassifier)
    print score5
    score6 = kFoldCrossVal(XTrain, YTrain, classifyMultinomialNaiveBayesClassifier)
    print score6
    
    scoreOneVsRestClassifier.append(score)
    scoreOneVsOneClassifier.append(score1)
    scoreOutputCodeClassifier.append(score2)
    scoreRandomForestClassifier.append(score3)
    scoreGaussianNaiveBayesClassifier.append(score4)
    scoreBernoulliNaiveBayesClassifier.append(score5)
    scoreMultinomialNaiveBayesClassifier.append(score6)

In [96]:
# Replace missing values (NaN) with Zero
PreProcessClassificationScores('zeros')

33.7542277339
82.2096956032
67.5310033822
8.95152198422
54.4983089064


In [97]:
# Replace missing values (NaN) with mean values for each column
PreProcessClassificationScores('colmean')

59.5490417136
82.6606538895
77.6324689966
5.20856820744
65.5918827508


In [98]:
# Replace missing values (NaN) with median values for each column
PreProcessClassificationScores('colmedian')

55.8962795941
83.1341600902
77.6550169109
5.20856820744
64.9605411499


In [99]:
# Replace missing values (NaN) with mean values for each row
PreProcessClassificationScores('rowmean')

55.7609921082
82.9312288613
76.2570462232
5.20856820744
66.9898534386


In [100]:
# Replace missing values (NaN) with median values for each row
PreProcessClassificationScores('rowmedian')

65.321307779
82.2547914318
76.1894024803
5.20856820744
66.5839909808


In [101]:
# Replace missing values (NaN) with row spectral mean
PreProcessClassificationScores('rowspectralmean')

62.6606538895
81.9616685457
76.3021420519
5.20856820744
65.6144306652


In [102]:
# Replace missing values (NaN) with row spectral median
PreProcessClassificationScores('rowspectralmedian')

72.4464487035
83.3596392334
76.099210823
5.20856820744
65.7497181511


In [103]:
# Replace missing values (NaN) with min value for each row
PreProcessClassificationScores('colmin')

59.075535513
81.6910935738
72.8748590755
5.20856820744
60.7440811725


In [104]:
# Replace missing values (NaN) with min value for each row
PreProcessClassificationScores('colmax')

41.8263810598
81.6910935738
74.4532130778
5.20856820744
56.5501691094


In [105]:
# Replace missing values (NaN) with min value for each row
PreProcessClassificationScores('rowmin')

59.9098083427
81.1724915445
75.6257046223
5.20856820744
66.0653889515


In [106]:
# Replace missing values (NaN) with max value for each row
PreProcessClassificationScores('rowmax')

51.2965050733
82.2547914318
72.266065389
5.20856820744
63.1116121759


In [107]:
# Replace missing values (NaN) with 127
PreProcessClassificationScores('mid')

44.2841037204
81.8940248027
68.8613303269
5.20856820744
58.1510710259


In [108]:
# Replace missing values (NaN) with row spectral min
PreProcessClassificationScores('rowspectralmin')

63.8556933484
81.8940248027
76.5050732807
5.20856820744
65.2536640361


In [109]:
# Replace missing values (NaN) with row spectral max
PreProcessClassificationScores('rowspectralmax')

64.396843292
83.1116121759
73.4385569335
5.20856820744
63.2468996618


In [10]:
def writePrdictedLabelFile(YPred):
    f = open("Predictions.csv","w")
    f.write("Id,Prediction" + "\n")

    for i in xrange(len(YPred)):
        f.write(str(i+1) + "," + str(int(YPred[i]))+ "\n")
    
    f.close()

In [123]:
replacment = 'rowspectralmin'
classifierName = classifyGaussianNaiveBayesClassifier
XTrain, YTrain, rawTestData = readInputFiles()
XTrain = replaceMissingValues(XTrain, replacment)
XTest = replaceMissingValues(rawTestData, replacment)
score = kFoldCrossVal(XTrain, YTrain, classifierName)
print score
#YPred = OneVsOneClassifier(LinearSVC(random_state=0)).fit(XTrain, YTrain).predict(XTest)
#YPred = RandomForestClassifier(n_estimators=100).fit(XTrain, YTrain).predict(XTest)
YPred = GaussianNB().fit(XTrain, YTrain).predict(XTest)

writePrdictedLabelFile(YPred)

76.5050732807


In [120]:
#print scoreOneVsRestClassifier
print scoreOneVsOneClassifier
#print scoreOutputCodeClassifier
print scoreRandomForestClassifier
print scoreGaussianNaiveBayesClassifier
print scoreBernoulliNaiveBayesClassifier
print scoreMultinomialNaiveBayesClassifier

[33.75422773393461, 59.549041713641486, 55.896279594137546, 55.76099210822999, 65.32130777903043, 62.66065388951522, 72.44644870349494, 59.075535512965054, 41.82638105975197, 59.9098083427283, 51.29650507328072, 44.28410372040586, 63.85569334836528, 64.39684329199548]
[82.20969560315672, 82.66065388951522, 83.13416009019166, 82.93122886133031, 82.25479143179255, 81.96166854565953, 83.35963923337093, 81.69109357384443, 81.69109357384443, 81.17249154453214, 82.25479143179255, 81.89402480270576, 81.89402480270576, 83.11161217587373]
[67.53100338218715, 77.63246899661782, 77.65501691093574, 76.25704622322436, 76.18940248027057, 76.3021420518602, 76.09921082299887, 72.87485907553551, 74.45321307779031, 75.62570462232243, 72.26606538895152, 68.86133032694475, 76.50507328072153, 73.43855693348365]
[8.951521984216459, 5.208568207440812, 5.208568207440812, 5.208568207440812, 5.208568207440812, 5.208568207440812, 5.208568207440812, 5.208568207440812, 5.208568207440812, 5.208568207440812, 5.20856